In [ ]:
require("pacman")
pacman::p_load(tidyverse, MatchIt, lmtest, sandwich, dplyr)

# Abrir base

In [53]:
base <- read.table(
        "./data/microdados_enade_2018.txt",
        header = TRUE, sep=";", dec = ",", 
        colClasses=c(DS_VT_ACE_OFG="character",
                        DS_VT_ACE_OCE="character"),
        stringsAsFactors=TRUE
    )

In [57]:
baseLimpa <- filter(base, NT_CE > 1 )

# Preparar variáveis

In [60]:
baseLimpa$isBolsistaPerm <- ifelse(baseLimpa$QE_I12 == "A", 0, 1)

In [61]:
model1 <- c(
    "CO_GRUPO", # curso
    "CO_MODALIDADE", # modalidade do curso (presencial/a distancia)
    "CO_UF_CURSO", # UF doc curso
    "CO_CATEGAD", # categoria adm
    "TP_SEXO", # sexo do aluno
    "NU_IDADE", # idade do aluno
    "QE_I01", # estado civil
    "QE_I02", # raça
    "QE_I04", # escolarização pai
    "QE_I05", # escolarização mãe
    "QE_I08", # renda familiar
    "NT_CE" # Nota componente específico
    # "isBolsistaPerm" # se o aluno teve ou não bolsa permanência
)

# Descritivas

# Realizar o matching

In [62]:
match_obj <- matchit(
    reformulate(model1, response = 'isBolsistaPerm'),
    data = baseLimpa, method = "nearest", distance ="glm",
    ratio = 1,
    replace = FALSE
)
summary(match_obj)


Call:
matchit(formula = reformulate(model1, response = "isBolsistaPerm"), 
    data = baseLimpa, method = "nearest", distance = "glm", replace = FALSE, 
    ratio = 1)

Summary of Balance for All Data:
              Means Treated Means Control Std. Mean Diff. Var. Ratio eCDF Mean
distance             0.1789        0.0356          0.4472   168.8680    0.2337
CO_GRUPO            73.4506       64.4602          0.0479     1.1581    0.0219
CO_MODALIDADE        1.0754        1.1707         -0.3608     0.4927    0.0476
CO_UF_CURSO         33.3478       34.5008         -0.1258     1.1156    0.0301
CO_CATEGAD        8111.3604     8011.4408          0.0256     0.9545    0.0729
TP_SEXOF             0.6191        0.6095          0.0198          .    0.0096
TP_SEXOM             0.3809        0.3905         -0.0198          .    0.0096
NU_IDADE            28.4479       29.0645         -0.0780     0.9280    0.0096
QE_I01               0.1311        0.0000          0.3885          .    0.1311
QE_I01A

#  Regressão

In [63]:
#Extract the matched data and save the data into the variable matched_data
matched_data <- match.data(match_obj)
#Run regression model with psychological distress as the outcome, and smoker as the only predictor
#We need to specify the weights - Matched participants have a weight of 1, unmatched participants 
res <- lm(NT_CE ~ isBolsistaPerm, data = matched_data, weights = weights)

In [64]:
#Test the coefficient using cluster robust standard error
coeftest(res, vcov. = vcovCL, cluster = ~subclass)


t test of coefficients:

               Estimate Std. Error t value  Pr(>|t|)    
(Intercept)    45.84036    0.11796 388.624 < 2.2e-16 ***
isBolsistaPerm -3.35103    0.16106 -20.807 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [65]:
#Calculate the confidence intervals based on cluster robust standard error
coefci(res, vcov. = vcovCL, cluster = ~subclass, level = 0.95)

,2.5 %,97.5 %
(Intercept),45.609168,46.071560
isBolsistaPerm,-3.666703,-3.035355
